# Functionality of this notebook: 

1. calculate the ciCOH for normal and mild LFP data in beta1 and beta2 band

2. stk is short for single target kluver


In [1]:
import os, sys
import scipy.io as sio
import glob
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

""" 
    user defined module
"""

# import_nbmodule used for import package in .ipynb
import import_nbmodule

# add path '../'
sys.path.append(os.path.abspath(os.path.join('..')))

# import ciCoherence_overtime in ../connAnalyTool/synchronization_indices.ipynb
from connAnalyTool.synchronization_indices import ciCoherence_acrosstrials
from connAnalyTool.synchronization_indices import ciCoherence_overtime


# import animal_folder in ../util/folder_extract.pynb 
from util.folder_extract import animal_folder

importing Jupyter notebook from /home/lingling/yang7003@umn.edu/NMRC_umn/Codes/FCAnalysis/connAnalyTool/synchronization_indices.ipynb
importing Jupyter notebook from /home/lingling/yang7003@umn.edu/NMRC_umn/Codes/FCAnalysis/util/folder_extract.ipynb


## calculate ciCOH

### function ciCOH_lfp <a id='funcciCOH_lfp'></a>

1. calculate ciCOH of each pair of channels in return phase from lfp data

2. only analyze the return phase whose duration is longer than 500ms and shorter than 1s

In [2]:
def ciCOH_lfp(lfp, fs, idxevent, idxevent_varNames, tdur_min = 0.5, tdur_max = 1):
    
    """
        return the ciCOH in the return phase for all the trials extracted from lfp data
        
        Args:
        
            lfp: local field potential data (nchns * ntemporals * ntrials)
            
            fs: sample rate
            
            idxevent: event indices array (ntrials *5)
            
            idxevent_varNames: event names array ['TargetTime' 'ReachTimeix' 'TouchTimeix' 'ReturnTimeix' 'MouthTimeix']
            
            tdur_min, tdur_max: minimal and maximal duration for return phase
            
        Return:
            ciCOH_alltrials: ciCOH values for all trials extracted from lfp data (nchns * nchns * ntrials)
    """


    #####################################################################
    #
    #    extract column index for event return [ReturnTimeix MouthTimeix]
    #
    #####################################################################

    # column index for event return
    coli_return = [i for i, s in enumerate(idxevent_varNames) if 'ReturnTimeix' in s]

    # column index for event Touch
    coli_mouth = [i for i, s in enumerate(idxevent_varNames) if 'MouthTimeix' in s]

    # column indices of return phase 
    cols_phasereturn = coli_return + coli_mouth

    cols = cols_phasereturn



    ###################################################
    #
    #    extract lfp data of return phase for each trial
    #
    ###################################################

    # idxdur_min, idxdur_max
    idxdur_min, idxdur_max = int(np.round(tdur_min * fs)), int(np.round(tdur_max * fs))

    nchns,_ ,ntrials = lfp.shape
    
    # initial ciCOH_alltrials array for all trials
    ciCOH_alltrials = np.zeros((nchns, nchns, ntrials))

    for triali in range(ntrials):

        ### extract the extract lfp data of return phase for each trial, only extract that last longer than 500ms ###

        # start and end indices over time of specific phase for triali 
        idxtime_str, idxtime_end = int(idxevent[triali, cols[0]]), int(idxevent[triali, cols[1]])

        # remove trials of return phase longer than tdur_max and shorter than tdur_min
        if idxtime_end - idxtime_str < idxdur_min or idxtime_end - idxtime_str > idxdur_max:
            continue;

        # extract lfp data in specific phase of one trial (n_chns * n_times)
        lfp_1trial = lfp[:, idxtime_str:idxtime_end, triali]


        ### calculate ciCOH between each pair of channels ###

        for chni in range(nchns-1):

            # signal of channel chni
            signal1 = lfp_1trial[chni, :]

            for chnj in range(chni+1, nchns):

                # signal of channel chnij
                signal2 = lfp_1trial[chnj, :]

                # ciCOH_alltrials assignment
                ciCOH_alltrials[chni, chnj, triali] =  ciCoherence_overtime(signal1, signal2)
                
                # symmetrical
                ciCOH_alltrials[chnj, chni, triali] = ciCOH_alltrials[chni, chnj, triali]

                del signal2
            del signal1

        del idxtime_str, idxtime_end, lfp_1trial


    # return
    return ciCOH_alltrials

### function ciCOH1file <a id = 'func_ciCOH1file'></a>
1. calculate ciCOH of each pair of channels in return phase for one file

2. call the function [ciCOH_lfp](#funcciCOH_lfp)

In [3]:
def ciCOH_1file(file):
    
    """
        ciCOH values of beta1 and beta2 band in the return pahse for 1 file
        
        Args:
        
        file: file name 
        
        Returns:
        
            ciCOH_beta1_1file: 
                beta1 band corrected imaginary coherence value of all pairs of channels for all trials in file 
                (nchns * nchns * ntrials)
                         
            ciCOH_beta2_1file: 
                beta2 band corrected imaginary coherence value of all pairs of channels for all trials in file 
                (nchns * nchns * ntrials)
    """
    

    ######################################
    #
    #    extract data of lfp, idxevent, fs
    #
    ######################################
    
    # load the stk trial data in beta1/beta2 band with idxevent, idxevent_varNames, fs
    matdat = sio.loadmat(file, variable_names = ['lfptrial_beta1', 'lfptrial_beta2', 'idxevent', 'idxevent_varNames', 'fs'], struct_as_record = False, squeeze_me = True)

    # fs: sample rate
    fs = matdat['fs'] 

    # idxevent (n_trials * 5): indices for targetonset, reachonset, reach, return and mouth
    idxevent, idxevent_varNames = matdat['idxevent'], matdat['idxevent_varNames']
        
    # lfp_beta1, and lfp_beta2
    lfp_beta1, lfp_beta2 = matdat['lfptrial_beta1'], matdat['lfptrial_beta2'],
    
    # ciCOH values of beta1 band in the return pahse for 1 file
    ciCOH_beta1_1file = ciCOH_lfp(lfp_beta1, fs, idxevent, idxevent_varNames, tdur_min = 0.5, tdur_max = 1)
    
    # ciCOH values of beta2 band in the return pahse for 1 file
    ciCOH_beta2_1file = ciCOH_lfp(lfp_beta2, fs, idxevent, idxevent_varNames, tdur_min = 0.5, tdur_max = 1)
        
        
    # return
    return ciCOH_beta1_1file, ciCOH_beta2_1file

### function ciCOH_allfiles

1. return a ciCOH dictionary result containing concatenated ciCOH values from all normal and mild files individually
   in beta1 and beta2 bands

2. call function [ciCOH_1file(file)](#func_ciCOH1file)

In [4]:
def ciCOH_allfiles(files):
    
    """
        return a ciCOH dictionary result containing concatenated ciCOH values from all normal and mild files individually
        in beta1 and beta2 bands 
        
        
        Return:
            dic_ciCOH: a ciCOH dictionary result 
                       dic_ciCOH.keys() = ['ciCOH_beta1_normal', 'ciCOH_beta1_mild', 'ciCOH_beta2_normal', 'ciCOH_beta2_mild']
    """
    
    

    # delete ciCOH_beta1_normal if exist inside this function
    if 'ciCOH_beta1_normal' in locals():
        del ciCOH_beta1_normal
        
    # delete ciCOH_beta1_mild if exist inside this function
    if 'ciCOH_beta1_mild' in locals():
        del ciCOH_beta1_mild
        
        
    # delete ciCOH_beta2_normal if exist inside this function
    if 'ciCOH_beta2_normal' in locals():
        del ciCOH_beta2_normal
        
    # delete ciCOH_beta2_mild if exist inside this function
    if 'ciCOH_beta2_mild' in locals():
        del ciCOH_beta2_mild
    
    # concatenate ciCOH values from all files
    for file in files:
        
        print("dealing " + file)

        # ciCOH value of all pairs of channels for all trials in one file (nchns * nchns * ntrials)
        ciCOH_beta1_1file, ciCOH_beta2_1file = ciCOH_1file(file)

        # normal case
        if 'normal' in file and 'mild' not in file:
            
            # beta1 normal
            if 'ciCOH_beta1_normal' not in locals():
                # first normal file
                ciCOH_beta1_normal = ciCOH_beta1_1file
            else:
                # append normal file
                ciCOH_beta1_normal = np.concatenate((ciCOH_beta1_normal, ciCOH_beta1_1file), axis=2)
                
            # beta2 normal
            if 'ciCOH_beta2_normal' not in locals():
                # first normal file
                ciCOH_beta2_normal = ciCOH_beta2_1file
            else:
                # append normal file
                ciCOH_beta2_normal = np.concatenate((ciCOH_beta2_normal, ciCOH_beta2_1file), axis=2)

        # mild case
        if 'mild' in file and 'normal' not in file:
            
            # beta1 mild
            if 'ciCOH_beta1_mild' not in locals():
                # first mild file
                ciCOH_beta1_mild = ciCOH_beta1_1file
            else:
                # append mild file
                ciCOH_beta1_mild = np.concatenate((ciCOH_beta1_mild, ciCOH_beta1_1file), axis=2)
                
            # beta2 mild
            if 'ciCOH_beta2_mild' not in locals():
                # first mild file
                ciCOH_beta2_mild = ciCOH_beta2_1file
            else:
                # append mild file
                ciCOH_beta2_mild = np.concatenate((ciCOH_beta2_mild, ciCOH_beta2_1file), axis=2)

        del ciCOH_beta1_1file, ciCOH_beta2_1file
        
    
    # create ciCOH results dictionary
    dic_ciCOH = {"ciCOH_beta1_normal": ciCOH_beta1_normal, "ciCOH_beta1_mild": ciCOH_beta1_mild,
                 "ciCOH_beta2_normal": ciCOH_beta2_normal, "ciCOH_beta2_mild": ciCOH_beta2_mild} 
    
    return dic_ciCOH

## save ciCOH

func ciCOH_save: save calculated ciCOH to aniresfolder/res_ciCOH.pickle

In [5]:
import pickle

def ciCOH_save (dic_ciCOH, aniresfolder, savefilename):
    """
        save dic_ciCOH to aniresfolder/res_ciCOH.pickle
    """

    # file storing the ciCOH results
    ciCOHfile = os.path.join(aniresfolder, savefilename)

    pickle.dump(dic_ciCOH, open(ciCOHfile, "wb" ) )

## main function

In [6]:
def main(animal):
    # extract the data, result and metainf folder of animal
    anidatfolder, aniresfolder, animetafolder = animal_folder(animal = animal)

    # load data folder
    loaddatafolder = os.path.join(anidatfolder, 'epochs_filtered', 'betaBand')

    # glob all .mat files (both mild and normal files)
    files = glob.glob(os.path.join(loaddatafolder, '*.mat'))

    #  a ciCOH dictionary containing concatenated ciCOH values from all normal and mild files 
    # individually in beta1 and beta2 bands
    print("calculating ciCOH values in beta1 and beta2 bands for normal and mild individually.....")
    dic_ciCOH = ciCOH_allfiles(files)

    # save dic_ciCOH
    savefilename = 'res_ciCOH_beta.pickle'
    ciCOH_save (dic_ciCOH, aniresfolder, savefilename)

In [7]:
animal = 'Pinky'
main(animal)

calculating ciCOH values in beta1 and beta2 bands for normal and mild individually.....
dealing /home/lingling/yang7003@umn.edu/NMRC_umn/Projects/FCAnalysis/data/Pinky/epochs_filtered/betaBand/Pinky_lfptrial_normal_042017_bktdt5.mat
dealing /home/lingling/yang7003@umn.edu/NMRC_umn/Projects/FCAnalysis/data/Pinky/epochs_filtered/betaBand/Pinky_lfptrial_mild_101117_bktdt1.mat
dealing /home/lingling/yang7003@umn.edu/NMRC_umn/Projects/FCAnalysis/data/Pinky/epochs_filtered/betaBand/Pinky_lfptrial_normal_012417_bktdt2.mat
dealing /home/lingling/yang7003@umn.edu/NMRC_umn/Projects/FCAnalysis/data/Pinky/epochs_filtered/betaBand/Pinky_lfptrial_normal_020217_bktdt9.mat
dealing /home/lingling/yang7003@umn.edu/NMRC_umn/Projects/FCAnalysis/data/Pinky/epochs_filtered/betaBand/Pinky_lfptrial_normal_012617_bktdt2.mat
dealing /home/lingling/yang7003@umn.edu/NMRC_umn/Projects/FCAnalysis/data/Pinky/epochs_filtered/betaBand/Pinky_lfptrial_normal_020917_bktdt5.mat
dealing /home/lingling/yang7003@umn.edu/NMRC

## plot section

In [ ]:
plt.imshow(np.mean(ciCOH_mild, axis = 2));
plt.colorbar()
plt.title('mild')
plt.show()

In [ ]:
plt.imshow(np.mean(ciCOH_normal, axis = 2))
plt.title('normal')
plt.colorbar()
plt.show()

In [ ]:
print(np.mean(ciCOH_normal, axis = 2))

In [ ]:
print(np.mean(ciCOH_mild, axis = 2))

## Test section